In [2]:
from scrapling import Fetcher
page = Fetcher().get('http://www.bom.gov.au/cgi-bin/wrap_fwo.pl?IDQ60005.html', stealthy_headers=True, follow_redirects=True)

#print the type of the page
print(type(page))

#convert page to string
page = str(page)

[2025-01-15 08:18:18] INFO: Fetched (200) <GET http://www.bom.gov.au/cgi-bin/wrap_fwo.pl?IDQ60005.html> (referer: https://www.google.com/search?q=bom)


<class 'scrapling.engines.toolbelt.custom.Response'>


In [19]:
from bs4 import BeautifulSoup
import pandas as pd
import re

#convert page to string
page_str = str(page)

page_soup = BeautifulSoup(page_str, "html.parser")

# Find the table (assuming there's only one table in the HTML)
table = page_soup.find("table")

all_rows = []  # will hold lists of cell values

for tr in table.find_all("tr"):
    # Gather all cells (td/th) in this row
    cells = tr.find_all(["td", "th"])
    
    # Skip row if any cell has a colspan
    # (You could also check for rowspan if needed)
    skip_row = any(cell.has_attr("colspan") for cell in cells)
    if skip_row:
        continue

    # Extract the text from each cell in the row
    row_data = [cell.get_text(strip=True) for cell in cells]
    
    # You mention the table has 7 columns, so only keep rows that have 7 cells
    if len(row_data) == 7:
        all_rows.append(row_data)

    # -------------------------------------------------------------------------
    # 1) Look for the link that contains the word "plot" (case-insensitive)
    # -------------------------------------------------------------------------
    plot_link_tag = tr.find("a", string=lambda text: text and "plot" in text.lower())
    
    # If found, parse the link's href to extract your IDQ number and second number
    if plot_link_tag and plot_link_tag.has_attr("href"):
        href = plot_link_tag["href"]  # e.g. http://www.bom.gov.au/fwo/IDQ65388/IDQ65388.540612.plt.shtml
        
        # Get the filename after the last slash, e.g. "IDQ65388.540612.plt.shtml"
        filename = href.split("/")[-1]
        
        # Use a regular expression to capture the part before the first '.' and the next part
        # Pattern: ^(.*?)\.(.*?)\.plt\.shtml$
        match = re.match(r'^(.*?)\.(.*?)\.plt\.shtml$', filename)
        if match:
            idq_number = match.group(1)     # e.g. "IDQ65388"
            second_number = match.group(2)  # e.g. "540612"
        else:
            idq_number = None
            second_number = None
    else:
        # If there is no link or no href attribute
        idq_number = None
        second_number = None
    
    # -------------------------------------------------------------------------
    # 2) Append these two new values into row_data
    # -------------------------------------------------------------------------
    row_data.append(idq_number)
    row_data.append(second_number)
    
    #all_rows.append(row_data)

# Now `all_rows` has 7 + 2 = 9 columns total:
#   original 7 columns + IDQ_number + second_number

# If the first row in `all_rows` is your header, then:
header = all_rows[0]  # e.g. the original 7 column headers (if that's truly a header row)
# You might want to adjust for the new columns in the header, for example:
header = header + ["IDQ_Number", "Second_Number"]

# Assuming the first row is the header
df = pd.DataFrame(all_rows[1:], columns=all_rows[0])
# df = pd.DataFrame(all_rows[1:], columns=header)

# remove any instances of "^" from the height column
df['Height'] = df['Height'].str.replace('^', '')

# convert height column to float
# df['Height'] = df['Height'].astype(float)

print(df)

#export to csv
df.to_csv('flood.csv', index=False)


                             Station Name    Time/Day  Height Tendency  \
0     Lt Nerang Ck at Little Nerang Dam *  6.30am Wed  168.17   steady   
1     Lt Nerang Ck at Little Nerang Dam #  5.46am Wed    0.16   steady   
2           Nerang R at Numinbah Valley *  7.00am Wed    0.96   steady   
3           Nerang R at Numinbah Valley #  5.41am Wed    0.95   steady   
4                 Nerang R at Hinze Dam *  6.00am Wed   95.09   steady   
...                                   ...         ...     ...      ...   
1267                   Ugar Island Tide *  7.29am Wed    1.34   rising   
1268                   Iama Island Tide *  7.34am Wed    0.75   steady   
1269         Moa Island (St Pauls) Tide *  7.30am Wed    0.94   steady   
1270            Moa Island (Kubin) Tide *  7.32am Wed    2.40   steady   
1271               Thursday Island Tide *  7.35am Wed    1.66   steady   

                 Crossing  Flood Class Recent Data      None    None  
0     0.15 above Spillway               